# Explicación del Algoritmo NavGlobalPlanner

Este notebook explica y simula en Python el funcionamiento del `NavGlobalPlanner` (originalmente en C++). 
El objetivo es entender cómo procesa el archivo `global_plan.csv` para construir un grafo de navegación y planificar rutas.

## Pasos Principales:
1.  **Carga y Parseo**: Leer `global_plan.csv` e interpretar los diferentes tipos de datos (nodos del grafo, obstáculos, pozos).
2.  **Construcción del Grafo**: Crear un grafo dirigido donde los nodos son puntos en el espacio (x, y, orientación) y las aristas son las conexiones posibles.
3.  **Búsqueda de Rutas (A*)**: Implementar el algoritmo A* con la misma función de costo y heurística que usa el código C++.
4.  **Visualización**: Mostrar el mapa, el grafo y la ruta resultante.

## Clarificación de Roles: NavGlobalPlanner vs move_base Global Planner

Es importante distinguir entre `NavGlobalPlanner` y el planificador global utilizado por `move_base`.

1.  **NavGlobalPlanner (Mission Planner):**
    *   Este es el componente que estamos analizando en este notebook.
    *   Actúa como un **Gestor de Misiones** o planificador de alto nivel.
    *   Trabaja sobre un grafo estático predefinido en `global_plan.csv`.
    *   Su objetivo es encontrar la secuencia de nodos (waypoints) para ir de un punto A a un punto B en la mina, respetando la topología del grafo.
    *   No tiene en cuenta el mapa de costos local ni obstáculos dinámicos en tiempo real (aunque publica obstáculos estáticos para que otros los vean).
    *   Envía el camino global (lista de waypoints) al `NavigationServer`.

2.  **move_base Global Planner (`global_planner/GlobalPlanner`):**
    *   Es el planificador global estándar de ROS que utiliza `move_base`.
    *   Recibe los objetivos (waypoints) del `NavigationServer` (que a su vez los recibió de `NavGlobalPlanner`).
    *   Planifica una ruta detallada y libre de obstáculos en el mapa de costos global (`global_costmap`) para llegar al siguiente waypoint.
    *   Trabaja en conjunto con el planificador local (TEB) para la evasión de obstáculos y el control del robot.

**Flujo de Navegación:**
`NavGlobalPlanner` (Ruta en Grafo) -> `NavigationServer` (Orquestador) -> `move_base` (Planificación detallada y Control)

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math
from shapely.geometry import Point, Polygon, LineString
import heapq

## 1. Carga y Parseo del CSV

Cargamos el archivo `global_plan.csv`. Este archivo contiene toda la información estática necesaria para la navegación.

In [2]:
import ipywidgets as widgets
import io
import pandas as pd

print("Selecciona el archivo global_plan.csv desde tu equipo:")
uploader = widgets.FileUpload(
    accept='.csv',  # Aceptar solo archivos .csv
    multiple=False  # Solo un archivo
)
display(uploader)

Selecciona el archivo global_plan.csv desde tu equipo:


FileUpload(value=(), accept='.csv', description='Upload')

In [3]:
# Cargar el archivo seleccionado
if 'uploader' in locals() and uploader.value:
    try:
        # Manejar diferentes versiones de ipywidgets
        uploaded_file = None
        if isinstance(uploader.value, (tuple, list)):
            # ipywidgets >= 8.0
            uploaded_file = uploader.value[0]
        elif isinstance(uploader.value, dict):
            # ipywidgets < 8.0
            key = list(uploader.value.keys())[0]
            uploaded_file = uploader.value[key]
            
        if uploaded_file:
            print(f"Procesando archivo: {uploaded_file['name']}")
            content = uploaded_file['content']
            # 'content' son bytes
            df = pd.read_csv(io.BytesIO(content))
            print("Archivo cargado correctamente.")
            print(df.head())
        else:
            raise ValueError("Formato de archivo subido no reconocido.")
            
    except Exception as e:
        print(f"Error al leer el archivo subido: {e}")
        # Fallback a dummy data o error
        df = None
else:
    print("No se ha seleccionado ningún archivo en el widget anterior.")
    print("Intentando cargar archivo por defecto...")
    csv_path = './test_data/4_explain_nav_global_planner/global_plan.csv'
    try:
        df = pd.read_csv(csv_path)
        print(f"Cargado desde ruta por defecto: {csv_path}")
        print(df.head())
    except FileNotFoundError:
        print(f"No se encontró el archivo por defecto en {csv_path}.")
        # Crear un dataframe dummy si no existe el archivo para demostración
        data = {
            'type': ['graph_pose', 'graph_pose', 'graph_pose', 'hole'],
            'pose_type': ['home_pose', 'street', 'street', 'hole'],
            'graph_id': [0, 1, 2, 3],
            'drillhole_id': [-1, -1, -1, 101],
            'graph_pose_local': ['0,0,0', '10,0,0', '20,0,0', '20,5,1.57'],
            'connections': ['1', '0,2', '1', ''],
            'local_geometry': ['', '', '', 'POINT (20 5)']
        }
        df = pd.DataFrame(data)
        print("Usando datos de ejemplo.")

No se ha seleccionado ningún archivo en el widget anterior.
Intentando cargar archivo por defecto...
Cargado desde ruta por defecto: ./test_data/4_explain_nav_global_planner/global_plan.csv
   Unnamed: 0     z  z_hole  unkown0  unkown1  unkown2  unkown3  drillhole_id  \
0           0  11.8    10.0    270.0      0.0      0.0    -90.0          19.0   
1           1  11.2    10.0    270.0      0.0      0.0    -90.0          18.0   
2           2  30.0    10.0    270.0      0.0      0.0    -90.0          16.0   
3           3  25.0    10.0    270.0      0.0      0.0    -90.0          14.0   
4           4  11.6    10.0    270.0      0.0      0.0    -90.0          12.0   

   mesh                                      geometry  ...  \
0  peld   POINT (343348.51005739527 6334425.48609453)  ...   
1  peld  POINT (343353.35112230416 6334426.189037669)  ...   
2  peld  POINT (343358.49319460103 6334426.805178921)  ...   
3  peld   POINT (343363.4353070099 6334427.602388993)  ...   
4  peld   POI

## 2. Construcción del Grafo

Iteramos sobre el DataFrame para construir el grafo. 
- **Nodos**: Se crean a partir de `graph_pose_local` (x, y, theta).
- **Aristas**: Se crean a partir de la columna `connections`.

In [4]:
G = nx.DiGraph() # Grafo dirigido
pos = {}
node_data = {}
drillhole_to_node = {}

obstacles = []
holes = []

for index, row in df.iterrows():
    row_type = row['type']
    
    if row_type == 'graph_pose':
        # Parsear graph_pose_local "x,y,theta"
        try:
            pose_str = row['graph_pose_local'].replace('[', '').replace(']', '')
            parts = [float(x) for x in pose_str.split(',')]
            x, y, theta = parts[0], parts[1], parts[2]
            
            if pd.isna(row['graph_id']): continue
            node_id = int(row['graph_id'])
            
            # Agregar nodo al grafo
            G.add_node(node_id, x=x, y=y, theta=theta, type=row['pose_type'])
            pos[node_id] = (x, y)
            node_data[node_id] = {'x': x, 'y': y, 'theta': theta}

            # Detect Home Pose
            if str(row['pose_type']) == 'home_pose':
                drillhole_to_node['Home'] = node_id
            
            # Map drillhole_id to node_id if valid
            if 'drillhole_id' in row and pd.notna(row['drillhole_id']) and row['drillhole_id'] != -1:
                drillhole_to_node[int(row['drillhole_id'])] = node_id
            
            # Parsear conexiones
            connections_str = str(row['connections']).replace('[', '').replace(']', '')
            if connections_str and connections_str != 'nan':
                conn_parts = connections_str.split(',')
                for conn in conn_parts:
                    if conn.strip():
                        target_id = int(conn)
                        # Las aristas se agregan después para asegurar que todos los nodos existan
                        # pero en networkx se pueden agregar igual.
                        # Sin embargo, necesitamos calcular el peso después.
                        pass
        except Exception as e:
            print(f"Error parsing row {index}: {e}")
            
    elif row_type == 'hole':
        # Parsear geometría del pozo para visualización
        # Formato esperado: POINT (x y)
        geo_str = str(row['local_geometry'])
        if 'POINT' in geo_str:
            coords = geo_str.replace('POINT (', '').replace(')', '').split()
            holes.append((float(coords[0]), float(coords[1]), row['drillhole_id'], row.get('drillhole_order', -1)))

# Agregar aristas con pesos
# La función de peso en C++ es: dpos + dangle * angle_weight_ + 1.1
angle_weight = 1.0

def calculate_weight(n1, n2):
    dx = n1['x'] - n2['x']
    dy = n1['y'] - n2['y']
    dpos = math.sqrt(dx*dx + dy*dy)
    
    # Diferencia angular simplificada (en C++ usan cuaterniones)
    # Aquí asumimos theta en radianes
    # La lógica de C++ para dangle es: 2 * acos(dot_prod_quaternions)
    # Para 2D, podemos aproximar con la diferencia de ángulos
    
    # Replicando lógica de cuaterniones de C++:
    # q1 = (0, 0, sin(theta1/2), cos(theta1/2))
    # q2 = (0, 0, sin(theta2/2), cos(theta2/2))
    # dot = z1*z2 + w1*w2
    
    z1 = math.sin(n1['theta'] / 2)
    w1 = math.cos(n1['theta'] / 2)
    z2 = math.sin(n2['theta'] / 2)
    w2 = math.cos(n2['theta'] / 2)
    
    dot_prod = z1*z2 + w1*w2
    if dot_prod > 1.0: dot_prod = 0.9999
    if dot_prod < -1.0: dot_prod = -0.9999 # Safety
    
    dangle = 2 * math.acos(abs(dot_prod)) # abs porque q y -q son la misma rotación
    
    return dpos + dangle * angle_weight + 1.1

for index, row in df.iterrows():
    if row['type'] == 'graph_pose':
        source_id = int(row['graph_id'])
        connections_str = str(row['connections']).replace('[', '').replace(']', '')
        if connections_str and connections_str != 'nan':
            conn_parts = connections_str.split(',')
            for conn in conn_parts:
                if conn.strip():
                    target_id = int(conn)
                    if source_id in node_data and target_id in node_data:
                        weight = calculate_weight(node_data[source_id], node_data[target_id])
                        G.add_edge(source_id, target_id, weight=weight)

print("Pozos identificados (x, y, id):")
for h in holes:
    print(h)
print(f"Drillhole Mapping: {drillhole_to_node}")
print(f"Grafo construido con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")

Pozos identificados (x, y, id):
(38.77706003730418, 49.43914715666324, 19.0, 18.0)
(43.61812494619517, 50.14209029637277, 18.0, 17.0)
(48.76019724307116, 50.758231547661126, 16.0, 15.0)
(53.702309651940595, 51.55544161982834, 14.0, 12.0)
(58.6466603273293, 52.063076755963266, 12.0, 11.0)
(63.55331196618499, 52.61692895460874, 10.0, 8.0)
(42.83102895616321, 55.10657355282456, 17.0, 16.0)
(48.148283959017135, 55.992303433828056, 15.0, 14.0)
(53.37711448100163, 57.35784339811653, 13.0, 13.0)
(58.187319541932084, 57.233870416879654, 11.0, 10.0)
(63.224968008173164, 57.74179597757757, 9.0, 9.0)
(64.58760427095694, 47.62895135395229, 8.0, 6.0)
(57.0468575268751, 46.276366542093456, 6.0, 5.0)
(50.75048653228441, 46.31371960695833, 4.0, 3.0)
(43.84060640895041, 45.48965006694198, 2.0, 1.0)
(43.70646726246923, 37.891895518638194, 1.0, 0.0)
(57.583084242767654, 40.68468205444515, 5.0, 4.0)
(65.5513919054647, 40.94154519494623, 7.0, 7.0)
(51.270953457511496, 39.07002982869744, 3.0, 2.0)
Drillhole

## 3. Algoritmo A*

Implementamos A* usando la heurística de distancia Euclidiana, tal como en `NavGlobalPlanner.cpp`.

In [5]:
def heuristic(u, v):
    n1 = node_data[u]
    n2 = node_data[v]
    return math.sqrt((n1['x'] - n2['x'])**2 + (n1['y'] - n2['y'])**2)

def run_astar(start_node, goal_node):
    try:
        path = nx.astar_path(G, start_node, goal_node, heuristic=heuristic, weight='weight')
        return path
    except nx.NetworkXNoPath:
        return None
    except Exception as e:
        print(f"Error en A*: {e}")
        return None


## 4. Visualización

Visualizamos el grafo completo, los pozos y la ruta encontrada.

In [8]:
# Logic for Street Info (Ported from C++)

# Pre-filter street poses for efficiency
street_nodes = [n for n, d in G.nodes(data=True) if d.get('type') == 'street']
street_poses_list = [node_data[n] for n in street_nodes]

def get_street_info(robot_x, robot_y, robot_theta):
    # 1. Find nearest street pose
    min_dist = float('inf')
    nearest_node_id = None
    nearest_pose = None
    
    for nid in street_nodes:
        nd = node_data[nid]
        dist = math.sqrt((nd['x'] - robot_x)**2 + (nd['y'] - robot_y)**2)
        if dist < min_dist:
            min_dist = dist
            nearest_node_id = nid
            nearest_pose = nd
    
    if not nearest_pose:
        return None, None, None

    effective_nearest = nearest_pose.copy()
    if min_dist > 2.0:
        effective_nearest = {'x': robot_x, 'y': robot_y, 'theta': robot_theta}

    # 2. Calculate Recovery Pose (2 meters backwards from nearest/effective)
    rec_dist = -2.0
    rec_x = effective_nearest['x'] + rec_dist * math.cos(effective_nearest['theta'])
    rec_y = effective_nearest['y'] + rec_dist * math.sin(effective_nearest['theta'])
    
    recovery_pose = {'x': rec_x, 'y': rec_y, 'theta': effective_nearest['theta']}
    
    return nearest_pose, effective_nearest, recovery_pose

def visualize_interactive(start_k, end_k, robot_progress):
    if start_k not in drillhole_to_node or end_k not in drillhole_to_node:
        print("Selección inválida.")
        return

    start_n = drillhole_to_node[start_k]
    goal_n = drillhole_to_node[end_k]
    
    path = run_astar(start_n, goal_n)
    
    path_str_list = []
    if path:
        # Build string with ID and coords
        for nid in path:
            nx_node = pos[nid]
            path_str_list.append(f"{nid}({nx_node[0]:.1f}, {nx_node[1]:.1f})")
        
        print(f"Nodos del camino ({len(path)}): {path}")
        print(f"Poses Waypoints: {', '.join(path_str_list)}")
    else:
        print("No se encontró camino.")
    
    plt.figure(figsize=(12, 12))
    
    # Background: Nodes and Holes
    if holes:
        # Unpack 4 values now: x, y, id, order
        try:
            hx, hy, hids, horders = zip(*holes)
            plt.scatter(hx, hy, c='red', marker='x', alpha=0.5, label='Pozos')
            # Label with Order
            for x_h, y_h, id_h, order_h in holes:
                 order_str = str(int(order_h)) if pd.notna(order_h) else '?'
                 plt.text(x_h, y_h + 0.5, f"#{order_str}", fontsize=8, color='darkred', ha='center')
        except ValueError:
            pass
        
    gx, gy = zip(*pos.values())

    # Draw Nodes 
    plt.scatter(gx, gy, s=5, c='gray', alpha=1)

    if path:
        path_x = [pos[n][0] for n in path]
        path_y = [pos[n][1] for n in path]

        # Draw Ruta Global
        plt.plot(path_x, path_y, '--b', linewidth=1, alpha=0.5, label='Ruta Global')
        
        # Label path nodes with small indices
        for n_id in path:
             px, py = pos[n_id]
             plt.text(px, py + 0.5, str(n_id), fontsize=6, color='black', alpha=0.8)
        
        if len(path) > 1:
            total_len = 0
            segs = []
            for i in range(len(path)-1):
                p1 = pos[path[i]]
                p2 = pos[path[i+1]]
                dist = math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
                segs.append(dist)
                total_len += dist
            
            target_dist = total_len * robot_progress
            current_dist = 0
            robot_x, robot_y, robot_theta = pos[path[0]][0], pos[path[0]][1], 0
            
            for i, seg_len in enumerate(segs):
                if current_dist + seg_len >= target_dist:
                    remain = target_dist - current_dist
                    ratio = remain / seg_len if seg_len > 0 else 0
                    p1 = pos[path[i]]
                    p2 = pos[path[i+1]]
                    robot_x = p1[0] + (p2[0] - p1[0]) * ratio
                    robot_y = p1[1] + (p2[1] - p1[1]) * ratio
                    robot_theta = math.atan2(p2[1]-p1[1], p2[0]-p1[0])
                    break
                current_dist += seg_len
            else:
                robot_x, robot_y = pos[path[-1]]
                robot_theta = 0

            # Draw Robot 
            plt.plot(robot_x, robot_y, 'o', color='purple', markersize=10, label='Robot')
            plt.arrow(robot_x, robot_y, 1*math.cos(robot_theta), 1*math.sin(robot_theta), 
                      head_width=0.3, color='purple')
            
            nearest_orig, nearest_eff, recovery = get_street_info(robot_x, robot_y, robot_theta)
            
            if nearest_orig:

                #Draw Assigned Street
                plt.plot([robot_x, nearest_eff['x']], [robot_y, nearest_eff['y']], '--', color='orange', linewidth=2, label='Assigned Street')
                #Draw Nearest Street
                plt.plot(nearest_orig['x'], nearest_orig['y'], 's', color='cyan', markersize=8, label='Nearest Street Node')
                if nearest_eff != nearest_orig:
                     plt.plot(nearest_eff['x'], nearest_eff['y'], 'o', color='orange', markersize=4)

                # Draw Recovery
                plt.plot(recovery['x'], recovery['y'], '*', color='gold', markersize=12, label='Recovery Pose')
                plt.arrow(recovery['x'], recovery['y'], math.cos(recovery['theta']), math.sin(recovery['theta']), color='gold', head_width=0.5)
                
                # Print Values
                print(f"Robot Pose:     x={robot_x:.2f}, y={robot_y:.2f}")
                print(f"Nearest Pose:   x={nearest_orig['x']:.2f}, y={nearest_orig['y']:.2f}")
                print(f"Assigned Pose:  x={nearest_eff['x']:.2f}, y={nearest_eff['y']:.2f}")
                print(f"Recovery Pose:  x={recovery['x']:.2f}, y={recovery['y']:.2f}")

        # Draw Inicio
        plt.plot(pos[start_n][0], pos[start_n][1], 'go', markersize=8, label='Inicio')
        # Draw Meta
        plt.plot(pos[goal_n][0], pos[goal_n][1], 'm*', markersize=12, label='Meta')

    plt.title("Simulación NavGlobalPlanner + Street Info")
    plt.xlabel("X [m]")
    plt.ylabel("Y [m]")
    plt.legend(loc='upper right')
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.show()

# Widgets
style = {'description_width': 'initial'}
keys_raw = list(drillhole_to_node.keys()) if 'drillhole_to_node' in globals() else []
keys_str = sorted([k for k in keys_raw if isinstance(k, str)])
keys_int = sorted([k for k in keys_raw if isinstance(k, (int, float))])
drillhole_options = keys_str + keys_int

w_start = widgets.Dropdown(options=drillhole_options, description='Inicio:', style=style)
w_end = widgets.Dropdown(options=drillhole_options, description='Fin:', style=style)
if 'Home' in drillhole_options: w_start.value = 'Home'
if keys_int: w_end.value = keys_int[0]

w_robot_pos = widgets.FloatSlider(
    min=0.0, max=1.0, step=0.01, value=0.0,
    description='Robot Pos %:',
    continuous_update=True,
    layout=widgets.Layout(width='50%')
)

ui = widgets.VBox([
    widgets.HBox([w_start, w_end]),
    w_robot_pos
])

out = widgets.interactive_output(visualize_interactive, {
    'start_k': w_start, 
    'end_k': w_end, 
    'robot_progress': w_robot_pos
})

display(ui, out)


Output()